In [19]:
# Import necessary modules
from pyspark import SparkConf
from pyspark.streaming import StreamingContext
from pyspark.sql.session import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import *
from pyspark.sql.types import IntegerType, FloatType, StringType
from pyspark.ml import PipelineModel
from pyspark import SparkConf
from pyspark.streaming import StreamingContext
from pyspark.sql.session import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import *
from pyspark.sql.types import IntegerType, FloatType
from pyspark.ml import PipelineModel
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.pipeline import Pipeline
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.regression import RandomForestRegressor
import time

# Create a SparkSession
try:
    spark = SparkSession.builder.master("local[*]").getOrCreate()
    sc = spark.sparkContext
    streamingCont = StreamingContext(sc, 10)

    # Load the pre-trained model
    try:
        model_path = "CLUSTER_URL_HERE/model/model_rf_1687572265"
        model = PipelineModel.load(model_path)
        print("Loaded model..")
        
        
    except Exception as e:
        print(str(e))

    # Define a function to process each RDD
    def process(rdd):
        try:
            count = rdd.count()
            if count > 0:
                # Split the input data into a list of values
                rdd2 = rdd.map(lambda line: line.split(","))

                # Create a DataFrame from the input data
                df = rdd2.map(lambda line: Row(sqfeet=line[6],
                                beds=line[7],
                                baths=line[8],
                                cats_allowed=line[9],
                                dogs_allowed=line[10])).toDF()
                
                # cast the data types of the columns
                df = df.withColumn("sqfeet", df["sqfeet"].cast(IntegerType()))\
                       .withColumn("beds", df["beds"].cast(IntegerType()))\
                       .withColumn("baths", df["baths"].cast(IntegerType()))\
                       .withColumn("cats_allowed", df["cats_allowed"].cast(IntegerType()))\
                       .withColumn("dogs_allowed", df["dogs_allowed"].cast(IntegerType()))\
                
                df = df.dropna()
                print("---------------Houses info-------------")
                df.show()
                
                predictions = model.transform(df)
                # select the prediction column and show the result for each row
                results = predictions.select("prediction").collect()
                
                print("---------------Rental price-------------")
                print()
                for i, result in enumerate(results):    
                    print(f"House {i+1} is estimated to cost {result[0]:.2f} USD$")
                print()
                
                # Write the predictions to a file in HDFS
                output_path = "CLUSTER_URL_HERE/streaming/streaming_results/predictions.csv"
                predictions.select("prediction").write.mode("overwrite").csv(output_path)
                print(f"Predictions saved to {output_path}")

            else:
                print("Waiting for file..")
        except Exception as e:
            print(str(e))

    # Create a DStream from a text file
    dstream = streamingCont.textFileStream("CLUSTER_URL_HERE")
    dstream.foreachRDD(process)

    # Start the streaming context
    streamingCont.start()
    streamingCont.awaitTermination(timeout=1)

except Exception as e:
    print(str(e))

# Stop the streaming context and SparkSession
print("")
input('Press any key to continue')
streamingCont.stop()
spark.stop()
print("job ended")

Loaded model..

---------------Houses info-------------
+------+----+-----+------------+------------+
|sqfeet|beds|baths|cats_allowed|dogs_allowed|
+------+----+-----+------------+------------+
|   921|   2|    2|           0|           0|
|  1090|   2|    2|           0|           0|
|  1357|   3|    2|           1|           1|
|  1038|   2|    2|           0|           0|
|  1008|   2|    2|           1|           1|
|   640|   1|    1|           1|           1|
|   944|   2|    1|           1|           1|
|  1700|   3|    2|           1|           1|
|   780|   1|    1|           1|           1|
|   864|   2|    2|           0|           0|
+------+----+-----+------------+------------+

---------------Rental price-------------

House 1 is estimated to cost 1763.70$
House 2 is estimated to cost 1864.25$
House 3 is estimated to cost 2240.41$
House 4 is estimated to cost 1882.66$
House 5 is estimated to cost 1830.82$
House 6 is estimated to cost 1439.62$
House 7 is estimated to cost 

Press any key to continue a


job ended


In [ ]:
# Stop the StreamingContext
ssc.stop()